In [1]:
import os

from pyspark.sql import SparkSession
from pyspark.sql import functions as F

In [2]:
current_dir = os.getcwd()
dir_warehouse = f"{current_dir}/warehouse"

In [3]:
spark = SparkSession.builder \
    .appName("IcebergWithSpark") \
    .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions") \
    .config("spark.sql.catalog.hadoop_catalog", "org.apache.iceberg.spark.SparkCatalog") \
    .config("spark.sql.catalog.hadoop_catalog.type", "hadoop") \
    .config("spark.sql.catalog.hadoop_catalog.warehouse", dir_warehouse) \
    .config("spark.sql.default.catalog", "hadoop_catalog") \
    .getOrCreate()

25/01/07 21:14:34 WARN Utils: Your hostname, dell resolves to a loopback address: 127.0.1.1; using 192.168.15.6 instead (on interface wlp0s20f3)
25/01/07 21:14:34 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


25/01/07 21:14:34 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## Versões e Rollbacks de Tabelas

In [4]:
# Criação da Tabela
spark.sql("DROP TABLE IF EXISTS hadoop_catalog.default.vendas_versioned")

spark.sql("""
CREATE TABLE hadoop_catalog.default.vendas_versioned (
    id INT,
    produto STRING,
    quantidade INT,
    preco DOUBLE,
    data_venda DATE
)
USING iceberg
""")

DataFrame[]

In [5]:
# Dados
data_initial = [
    (1, "Produto A", 10, 15.5, "2023-11-01"),
    (2, "Produto B", 5, 22.0, "2023-11-02"),
    (3, "Produto C", 8, 30.0, "2023-11-03")
]
columns = ["id", "produto", "quantidade", "preco", "data_venda"]

df_initial = spark.createDataFrame(data_initial, columns)
df_initial = df_initial.withColumn("data_venda", F.to_date(F.col("data_venda"), "yyyy-MM-dd"))

df_initial.writeTo("hadoop_catalog.default.vendas_versioned").append()

In [6]:
# Visualização
spark.sql("SELECT * FROM hadoop_catalog.default.vendas_versioned").show()

+---+---------+----------+-----+----------+
| id|  produto|quantidade|preco|data_venda|
+---+---------+----------+-----+----------+
|  1|Produto A|        10| 15.5|2023-11-01|
|  2|Produto B|         5| 22.0|2023-11-02|
|  3|Produto C|         8| 30.0|2023-11-03|
+---+---------+----------+-----+----------+



In [7]:
# Criamos um novo snapshot inserindo mais dados
data_additional = [
    (4, "Produto D", 12, 25.0, "2023-11-04"),
    (5, "Produto E", 7, 18.5, "2023-11-05")
]
df_additional = spark.createDataFrame(data_additional, columns)
df_additional = df_additional.withColumn("data_venda", F.to_date(F.col("data_venda"), "yyyy-MM-dd"))

df_additional.writeTo("hadoop_catalog.default.vendas_versioned").append()

In [8]:
# Atualizamos preço
spark.sql("""
UPDATE hadoop_catalog.default.vendas_versioned
SET preco = 16.0
WHERE id = 1
""")

DataFrame[]

In [9]:
# Excluimos registro 2
spark.sql("""
DELETE FROM hadoop_catalog.default.vendas_versioned
WHERE id = 2
""")

DataFrame[]

In [10]:
# visualização
spark.sql("SELECT * FROM hadoop_catalog.default.vendas_versioned order by id").show()

+---+---------+----------+-----+----------+
| id|  produto|quantidade|preco|data_venda|
+---+---------+----------+-----+----------+
|  1|Produto A|        10| 16.0|2023-11-01|
|  3|Produto C|         8| 30.0|2023-11-03|
|  4|Produto D|        12| 25.0|2023-11-04|
|  5|Produto E|         7| 18.5|2023-11-05|
+---+---------+----------+-----+----------+



In [13]:
# Visualizamos os snapshots
snapshots_df = spark.sql("SELECT * FROM hadoop_catalog.default.vendas_versioned.snapshots")
snapshots_df.select("snapshot_id", "committed_at", "operation").show(truncate=False)
# snapshots_df.show(truncate=False)

+-------------------+-----------------------+---------+
|snapshot_id        |committed_at           |operation|
+-------------------+-----------------------+---------+
|7445524589099476635|2025-01-07 21:14:44.571|append   |
|6064031807186397825|2025-01-07 21:15:01.051|append   |
|4508017605226503077|2025-01-07 21:15:33.681|overwrite|
|3192364198481442317|2025-01-07 21:15:33.847|delete   |
+-------------------+-----------------------+---------+



In [14]:
# Obtemos o ID do snapshot para fazer rollback, indíce 1
snapshot_ids = snapshots_df.select("snapshot_id").orderBy("committed_at").collect()
rollback_snapshot_id = snapshot_ids[1].snapshot_id
print(f"Snapshot ID: {rollback_snapshot_id}")

Snapshot ID: 6064031807186397825


In [15]:
print("Antes do Rollback:")
spark.sql("SELECT * FROM hadoop_catalog.default.vendas_versioned order by id").show()

Antes do Rollback:
+---+---------+----------+-----+----------+
| id|  produto|quantidade|preco|data_venda|
+---+---------+----------+-----+----------+
|  1|Produto A|        10| 16.0|2023-11-01|
|  3|Produto C|         8| 30.0|2023-11-03|
|  4|Produto D|        12| 25.0|2023-11-04|
|  5|Produto E|         7| 18.5|2023-11-05|
+---+---------+----------+-----+----------+



In [16]:
# Rollback usado rollback_to_snapshot procedure
spark.sql(f"""
CALL hadoop_catalog.system.rollback_to_snapshot(
    table => 'hadoop_catalog.default.vendas_versioned',
    snapshot_id => {rollback_snapshot_id}
)
""")

ANTLR Tool version 4.9.3 used for code generation does not match the current runtime version 4.8ANTLR Runtime version 4.9.3 used for parser compilation does not match the current runtime version 4.8ANTLR Tool version 4.9.3 used for code generation does not match the current runtime version 4.8ANTLR Runtime version 4.9.3 used for parser compilation does not match the current runtime version 4.8

DataFrame[previous_snapshot_id: bigint, current_snapshot_id: bigint]

In [17]:
print("Dados depois do Rollback:")
spark.sql("SELECT * FROM hadoop_catalog.default.vendas_versioned order by id").show()


Dados depois do Rollback:
+---+---------+----------+-----+----------+
| id|  produto|quantidade|preco|data_venda|
+---+---------+----------+-----+----------+
|  1|Produto A|        10| 15.5|2023-11-01|
|  2|Produto B|         5| 22.0|2023-11-02|
|  3|Produto C|         8| 30.0|2023-11-03|
|  4|Produto D|        12| 25.0|2023-11-04|
|  5|Produto E|         7| 18.5|2023-11-05|
+---+---------+----------+-----+----------+

